<a href="https://colab.research.google.com/github/hwangsae91/project_so/blob/master/exploration/221007/%5BExp_07%5D20221007.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# # google colab전용
# from google.colab import drive
# drive.mount('/content/drive')

# exploration 7번째 과제</br>
@ 황한용(3기/쏘카)

## 라이브러리 선언

In [5]:
import pandas as pd
import konlpy
import gensim
import numpy as np

from collections import Counter
from konlpy.tag import Mecab
from pathlib import Path
from typing import Dict, Tuple


## 상수선언

In [6]:
DATA_PATH = "/".join([str(Path("./").parent.resolve().parent.resolve().parent.resolve().absolute()).replace("\\","/"),"data","sentiment_classification"]) # 데이터 기본경로
STEP_WORDS = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다'] # 불용어
MAX_NUM_WORDS = 10000 # 최대 사전 단어갯수
FEATURE_DATA = ["document"]
tokenizer = Mecab()


Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/

## 함수

In [ ]:
def load_data(
    train_data:pd.DataFrame
    , test_data:pd.DataFrame
    , num_words:int=MAX_NUM_WORDS
) -> Tuple[
            np.ndarray
            , np.ndarray
            , np.ndarray
            , np.ndarray
            , Dict[int,str]
        ]:
    """
    다음과 같은 전처리 후
    `train data(feature, target)`, `test data(feature, target)`, `단어사전`
    을 리턴한다.

    - 데이터의 중복 제거
    - `NaN` 결측치 제거
    - 한국어 토크나이저로 토큰화
    - 불용어(`stop_words`) 제거
    - 사전`word_to_index` 구성
    - 텍스트 스트링을 사전 인덱스 스트링으로 변환

    Parameters
    ----------
    train_data : DataFrame
        학습 데이터
    train_data : DataFrame
        테스트 데이터
    num_words : int, default = `MAX_NUM_WORDS`
        단어사전의 최대 단어 갯수

    Returns
    -------
    X_train : ndarray
        train feature data
    y_train : ndarray
        train target data
    X_test : ndarray
        test feature data
    y_test : ndarray
        test target data
    word_to_index : dict
        단어사전
    """
    # 1.
    train_data.drop_duplicates(subset=FEATURE_DATA, inplace=True)
    test_data.drop_duplicates(subset=FEATURE_DATA, inplace=True)

    # 2.
    train_data.dropna(how='any', inplace=True)
    test_data.dropna(how='any', inplace=True)

    tokenizer.morphs(train_data[FEATURE_DATA])


## 메인

In [ ]:
train_data = pd.read_table(DATA_PATH + "/ratings_train.txt")
test_data = pd.read_table(DATA_PATH + "/ratings_test.txt")
train_data.head()

데이터에 대한 설명은 아래와 같다.</br>
- id: The review id, provieded by Naver
- document: The actual review
- label: The sentiment class of the review. (0: negative, 1: positive)

In [ ]:
X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data)